In [1]:
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import cm
import mpl_toolkits.mplot3d.axes3d as p3
from matplotlib import colors

from scipy.optimize import approx_fprime, line_search, rosen, golden

import IPython, ipywidgets as widgets
from ipywidgets import interact, fixed

import warnings

%matplotlib inline

In [2]:
def calculate_angles(s):
    s_direction=np.copy(np.array(s))
    zero_indices=np.linalg.norm(s_direction,axis=1)>1e-5
    s_direction[zero_indices,:]=s_direction[zero_indices,:]/np.linalg.norm(s_direction[zero_indices,:],axis=1).reshape((-1,1))
    s_direction_dot=np.sum(s_direction[:-1,:]*s_direction[1:,:],axis=1)
    negative_indeces=s_direction_dot<0
    s_direction_dot[negative_indeces]=-s_direction_dot[negative_indeces]
    s_direction_dot=np.clip(np.abs(s_direction_dot), 0, 1)
    phi=np.arccos(s_direction_dot)*180/np.pi
    phi[negative_indeces]=180-phi[negative_indeces]
    return phi

In [3]:
plt.set_cmap(cm.twilight_shifted)
plt.set_cmap(cm.coolwarm)
# plt.set_cmap(cm.RdBu_r)
# plt.set_cmap(cm.gist_earth)

def plotOptimization(f, x, s, y, grads, a, xyrange=1.):
    
    # norm = colors.NoNorm()
    norm = colors.CenteredNorm()
    # norm = colors.TwoSlopeNorm(vcenter=0.)

    xx, yy = np.meshgrid(np.arange(-xyrange, xyrange, .01), 
                         np.arange(-xyrange, xyrange, .01))
    Z = f([xx,yy])
    
    fig, axs = plt.subplots(6,1, figsize=(10,20), height_ratios=[6,1,1,1,1,1])
    axs = axs.ravel()

    im=axs[0].pcolormesh(xx,yy,Z, norm=norm, alpha=.5, zorder=1)
    plt.colorbar(im)
    axs[0].contour(xx,yy,Z, levels=11, norm=norm, alpha=.75, zorder=3)
    axs[0].autoscale(enable=False, axis='both', tight=None)
    axs[0].set_aspect('equal')

    x = np.array(x).transpose()
    grads = np.array(grads)
    s = np.array(s)
    y = np.array(y).transpose()
    a = np.array(a)
    
    axs[0].scatter(x[0][0], x[1][0], marker='x', facecolor='r', s=100)
    axs[0].plot(x[0], x[1], 'g', marker='+')
    
    axs[1].set_title('Update history')
    axs[1].plot(s[:,0], s[:,1], label='$\\alpha_t s_t$')
    axs[1].legend()
    axs[1].set_aspect('equal')
    
    axs[2].plot(a, label='$\\alpha_t$')
    axs[2].set_title('learning rates')
    axs[2].legend()
    
    axs[3].set_title('Update step sizes')
    axs[3].plot(np.linalg.norm(s, ord=2, axis=-1), label='$|\\alpha_t s_t|$')
    axs[3].set_yscale('log')
    axs[3].legend()
    
    axs[4].set_title('Objective history')
    hdl1 = axs[4].plot(y, label='$f(x_t)$')
    ax2 = axs[4].twinx()
    hdl2 = ax2.plot(np.linalg.norm(grads, ord=2, axis=-1), c='r', label='$\\nabla f(x_t)$')
    ax2.set_yscale('log')
    
    axs[4].legend(handles = hdl1 + hdl2)
    
    phi=calculate_angles(s)
    
    axs[5].set_title('History of angle between directions')
    hdl3 = axs[5].plot(np.arange(1,phi.size+1),phi, label='$\\varphi_k$')    
    axs[5].set_ylim(0,180)
    axs[5].legend(handles = hdl3)
    
# when to terminate the iterations
def terminate(i, s, y):
    if i > 0:
        b = False
        
        # too tiny steps
        b = b or (np.linalg.norm(s[-1], ord=2) < 1e-9)
        
        # too little improvement
        b = b or (np.abs(y[-1] - y[-2]) < 1e-9)
        
        # too large values
        b = b or (y[-1] > 1e50)
        return b
    else:
        return False

<Figure size 640x480 with 0 Axes>

In [4]:
# some toy examples
functions = {
    'square':     lambda x_: x_[0]**2 + x_[1]**2, 
    'Rosenbrock': rosen,
    'sixhumps':   lambda x_: ((4 - 2.1*x_[0]**2 + x_[0]**4 / 3.) * x_[0]**2 + 
                              x_[0] * x_[1] + (-4 + 4*x_[1]**2) * x_[1] **2),
    'Himmelblau': lambda x_: (x_[0]**2 + x_[1] -11)**2 + (x_[0] + x_[1]**2 -7)**2,
    'Beale':      lambda x_:   (1.500 - x_[0] + x_[0]*x_[1])**2 
                             + (2.250 - x_[0] + x_[0]*x_[1]**2)**2
                             + (2.625 - x_[0] + x_[0]*x_[1]**3)**2,
    'Eggholder':  lambda x_:  -(x_[1] + .47)*np.sin(np.abs(x_[0]/2 + (x_[1] + .47)))
                             - x_[0] * np.sin(np.abs(x_[0] - (x_[1] + .47))),
    'Styblinski': lambda x_:   (x_[0]**4 - 16*x_[0]**2 + 5*x_[0]) 
                             - (x_[1]**4 - 16*x_[1]**2 + 5*x_[1]),
    'Matlab 1':   lambda x_: np.log(1 + 3*(x_[1] - (x_[0]**3 - x_[0]))**2 + (x_[0] - 4./3.)**2),
    'Matlab 2':   lambda x_: np.log(1 + 100. * (x_[0]**2 - x_[1])**2 + (1. - x_[1])**2)
}

In [5]:
# initial solution
x0 = [-.5,-0.1]

# Steepest Decent

In [6]:
@interact(f = widgets.ToggleButtons(
                options     = functions,
                description = 'Function $f$',
              ),
          x0 = fixed(x0),
          a = widgets.FloatLogSlider(value=1e-1,
                                     base=10, 
                                     min=-10, max=1, 
                                     step=.1, 
                                     continuous_update=False,
                                     description='learning rate $\\alpha$',
                                     style={'description_width':'initial'},
                                     readout_format='.1e'),
          max_iter = (1,15000, 1),
          xyrange = (1,5,1)
         )
def doSD(f, x0=[0,0], a=1e-2, max_iter=150, xyrange=1.):
    
    xt = x0
    at = 0.

    x = list()
    s = list()
    y = list()
    grads = list()
    
    for i in range(0,max_iter):

        dy = approx_fprime(xt, f, epsilon=1e-8)
        # dy /= np.linalg.norm(dy, ord=2)

        st = -dy
        at = a

        if i == 0:
            st *= 0

        xt = xt + at * st 
        yt = f(xt)

        grads.append(dy)
        s.append(st)
        x.append(xt)
        y.append(yt)

        if terminate(i, s, y):
            break
            
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        plotOptimization(f, x, s, y, grads, a=np.ones_like(y)*a, xyrange=xyrange)

interactive(children=(ToggleButtons(description='Function $f$', options={'square': <function <lambda> at 0x7fb…

## Stepest Descent with 1-D Line Search

In [7]:
@interact(f = widgets.ToggleButtons(
                options     = functions,
                description = 'Function $f$',
              ),
          x0 = fixed(x0),
          a = widgets.FloatLogSlider(value=1e-1,
                                      base=10, 
                                      min=-10, max=0, 
                                      step=.1, 
                                      continuous_update=False,
                                      description='learning rate $\\alpha$',
                                      style={'description_width':'initial'},
                                      readout_format='.1e'),
          max_iter = (1,15000, 1),
          xyrange = (1,5,1)
         )

def doSD(f, x0=[0,0], a=1e-2, max_iter=150, xyrange=1.):
    
    xt = x0
    at = 0.

    x = list()
    s = list()
    a = list()
    y = list()
    grads = list()
        
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        
        for i in range(0,max_iter):

            dy = approx_fprime(xt, f, epsilon=1e-8)
            # dy /= np.linalg.norm(dy, ord=2)

            st = -dy

            if i == 0:
                st *= 0
            else:
                #at = line_search(f, lambda x_: approx_fprime(x_, f), xt, st)[0] or a
                unit_directions=st/np.linalg.norm(st)
                g_line=lambda alpha: f(xt+alpha*unit_directions)
                at = golden(g_line,brack=(0,5))/np.linalg.norm(st)
            xt = xt + at * st 
            yt = f(xt)

            grads.append(dy)
            s.append(st)
            x.append(xt)
            y.append(yt)
            a.append(at)

            if terminate(i, s, y):
                break

        plotOptimization(f, x, s, y, grads, a, xyrange=xyrange)


interactive(children=(ToggleButtons(description='Function $f$', options={'square': <function <lambda> at 0x7fb…

## Conjugate Gradients

In [8]:
@interact(f = widgets.ToggleButtons(
                options     = functions,
                description = 'Function $f$',
              ),
          x0 = fixed(x0),
          max_iter = (1,1500, 1),
          xyrange = (1,5,1)
         )
def doCG(f, x0=[0,0], max_iter=150, xyrange=1.):
    
    xt = x0
    at = 0.

    x = list()
    s = list()
    y = list()
    grads = list()
    a = list()
    
    for i in range(0,max_iter):
        dy = approx_fprime(xt, f, epsilon=1e-8)
        
        if i > 1:
            g_0 = np.linalg.norm(dy, ord=2, axis=-1)**2
            g_1 = np.linalg.norm(grads[-1], ord=2, axis=-1)**2
            bt = g_0 / g_1
            st = -dy + bt * s[-1]
            
        else:
            st = -dy
            
        if i == 0:
            st *= 0
        else:           
            #at = line_search(f, lambda x_: approx_fprime(x_, f), xt, st)[0] or 1e-1
            unit_directions=st/np.linalg.norm(st)
            g_line=lambda alpha: f(xt+alpha*unit_directions)
            at = golden(g_line,brack=(0,5))/np.linalg.norm(st)
            
        xt = xt + at * st
        yt = f(xt)

        grads.append(dy)
        s.append(st)
        x.append(xt)
        y.append(yt)
        a.append(at)

        if terminate(i, s, y):
            break
            
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        plotOptimization(f, x, s, y, grads, a, xyrange)

interactive(children=(ToggleButtons(description='Function $f$', options={'square': <function <lambda> at 0x7fb…